Code für Blocking mit Intervall + Key (publisher)

In [2]:
import blocking_hash as hash 
import blocking_ngram as ngram
import blocking_structured_and_sort as ss
import matchers as m
import similarity as sim
import csv

def apply_similarity(df1, df2, threshold, selected_columns, similarity_function):
    similar_pairs = []

    for index1, row1 in df1.iterrows():
        for index2, row2 in df2.iterrows():
            average_similarity = 0.0

            for col_df1, col_df2 in zip(selected_columns, selected_columns):
                value_df1 = set(row1[col_df1].lower().split())
                value_df2 = set(row2[col_df2].lower().split())

                similarity = similarity_function(value_df1, value_df2)
                average_similarity += similarity

            if len(selected_columns) > 1:
                average_similarity /= len(selected_columns)

            if average_similarity >= threshold:
                pair = (row1[selected_columns].to_dict(), row2[selected_columns].to_dict(), average_similarity)
                similar_pairs.append(pair)

    return similar_pairs



def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    for key1, block1 in blocks1.items():
        for key2, block2 in blocks2.items():
            average_similarity = 0.0

            value_block1 = [block1.get(i, '') for i in indices]
            value_block2 = [block2.get(i, '') for i in indices]
            similarity = similarity_function(value_block1, value_block2)

            average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity > threshold:
                index_pairs = [(i, j) for i in block1['index'] for j in block2['index']]
                similar_pairs.extend(index_pairs)
    return similar_pairs

def similar_pairs_to_csv(similar_pairs, output_csv_file):
    header = ['dblp_index', 'acm_index']
    with open(output_csv_file, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        for pair in similar_pairs:
            writer.writerow(pair)

In [3]:
import pandas as pd
dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)

acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)

In [5]:
import time

def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    for key1, block1 in blocks1.items():
        for key2, block2 in blocks2.items():
            average_similarity = 0.0

            value_block1 = [block1.get(i, '') for i in indices]
            value_block2 = [block2.get(i, '') for i in indices]
            similarity = similarity_function(value_block1, value_block2)

            average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity > threshold:
                index_pairs = [(i, j) for i in block1['index'] for j in block2['index']]
                similar_pairs.extend(index_pairs)

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs


In [3]:
threshold = 0.7
n = 2
selected_indices = ['ngram_values']

selected_columns = ['author_names', 'paper_title']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity_ngrams, selected_indices)

In [5]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    for block1 in blocks1:
        for block2 in blocks2:
            average_similarity = 0.0

            value_block1 = [item.get(i, '') for item in block1 for i in indices]
            value_block2 = [item.get(i, '') for item in block2 for i in indices]
            similarity = similarity_function(value_block1, value_block2)

            average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity > threshold:
                index_key1 = [item.get('index', '') for item in block1]
                index_key2 = [item.get('index', '') for item in block2]

                if index_key1 and index_key2:
                    index_pairs = [(i, j) for i in index_key1 for j in index_key2]
                    similar_pairs.extend(index_pairs)

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs
# if no key

Matching with Initial n-gram (several combinations) with n =2,3 and treshold = 0.7, 0.85

In [8]:
threshold = 0.7
n = 2
selected_indices = ['ngram_values']

selected_columns = ['author_names', 'paper_title']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ay = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_py = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

Processing time: 20.790218830108643 seconds. Number of similar pairs: 1775


KeyboardInterrupt: 

In [35]:
threshold = 0.85
n = 2
selected_indices = ['ngram_values']

selected_columns = ['author_names', 'paper_title']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ay = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_py = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

Processing time: 18.389568090438843 seconds. Number of similar pairs: 1550
Processing time: 14.316695213317871 seconds. Number of similar pairs: 2597
Processing time: 12.704833030700684 seconds. Number of similar pairs: 9119
Processing time: 22.55008101463318 seconds. Number of similar pairs: 1606
Processing time: 22.843865871429443 seconds. Number of similar pairs: 1644
Processing time: 17.630688905715942 seconds. Number of similar pairs: 6099
Processing time: 16.811478853225708 seconds. Number of similar pairs: 1767
Processing time: 16.732720851898193 seconds. Number of similar pairs: 2058
Processing time: 19.991926908493042 seconds. Number of similar pairs: 2055
Processing time: 27.35147714614868 seconds. Number of similar pairs: 1680


In [36]:
threshold = 0.7
n = 3
selected_indices = ['ngram_values']

selected_columns = ['author_names', 'paper_title']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ay = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_py = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

Processing time: 18.19693899154663 seconds. Number of similar pairs: 1620
Processing time: 13.01272177696228 seconds. Number of similar pairs: 3897
Processing time: 12.109666109085083 seconds. Number of similar pairs: 29923
Processing time: 20.61672616004944 seconds. Number of similar pairs: 1670
Processing time: 21.168354272842407 seconds. Number of similar pairs: 1729
Processing time: 20.855627059936523 seconds. Number of similar pairs: 50182
Processing time: 15.7484769821167 seconds. Number of similar pairs: 1917
Processing time: 15.808166980743408 seconds. Number of similar pairs: 3927
Processing time: 18.846152782440186 seconds. Number of similar pairs: 6446
Processing time: 25.752315998077393 seconds. Number of similar pairs: 1806


In [37]:
threshold = 0.85
n = 3
selected_indices = ['ngram_values']

selected_columns = ['author_names', 'paper_title']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ay = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_apvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_py = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['paper_title','publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_ppvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue', 'year']
db = ngram.initial_ngram(dblp, n, selected_columns)
am = ngram.initial_ngram(acm, n, selected_columns)
sim_appvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_indices)

Processing time: 17.519720315933228 seconds. Number of similar pairs: 1368
Processing time: 13.078917980194092 seconds. Number of similar pairs: 1881
Processing time: 12.237350940704346 seconds. Number of similar pairs: 4407
Processing time: 20.92912197113037 seconds. Number of similar pairs: 1431
Processing time: 21.51428985595703 seconds. Number of similar pairs: 1467
Processing time: 17.251665830612183 seconds. Number of similar pairs: 3597
Processing time: 15.821579933166504 seconds. Number of similar pairs: 1639
Processing time: 15.934384822845459 seconds. Number of similar pairs: 1837
Processing time: 18.93509292602539 seconds. Number of similar pairs: 1860
Processing time: 25.655596256256104 seconds. Number of similar pairs: 1556


Matching with ngram without intials n = 2,3 and treshold = 0.7, 0.85

In [9]:
threshold = 0.
n = 2

dblp['year'] = dblp['year'].astype(str)
acm['year'] = acm['year'].astype(str)

selected_columns = ['author']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_a = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['paper_title']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_p = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'paper_title']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_ay = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'publication_venue']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_apv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'paper_title', 'year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_apy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_appv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'publication_venue', 'year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_apvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['paper_title','year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_py = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['paper_title','publication_venue']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_ppv = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['paper_title','publication_venue', 'year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_ppvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author_names', 'paper_title', 'publication_venue', 'year']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_appvy = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

# took to long 
# even with low treshold not many matches and most matches when 2 pairs

Processing time: 96.30731892585754 seconds. Number of similar pairs: 3664
Processing time: 44.41570806503296 seconds. Number of similar pairs: 8355
Processing time: 74.83391523361206 seconds. Number of similar pairs: 5734
Processing time: 102.37928795814514 seconds. Number of similar pairs: 1680
Processing time: 130.99481391906738 seconds. Number of similar pairs: 525
Processing time: 85.29151010513306 seconds. Number of similar pairs: 629
Processing time: 56.8927481174469 seconds. Number of similar pairs: 6197
Processing time: 91.67022490501404 seconds. Number of similar pairs: 29035
Processing time: 97.49070572853088 seconds. Number of similar pairs: 539
Processing time: 148.30595111846924 seconds. Number of similar pairs: 0


In [12]:
dblp_csv = '../CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = '../CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)
threshold = 0.7

selected_indices = ['hash_values']
selected_columns = ['author_names', 'paper_title']

db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ap =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)
print(len(similairty_ap))

selected_columns = ['author_names', 'paper_title','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_apy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_appv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['author_names', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_apvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_appvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)



selected_columns = ['paper_title', 'publication_venue']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ppv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['paper_title', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ppvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


1235


In [24]:
dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)

threshold = 0.85

selected_indices = ['ngram_values']
selected_columns = ['author_names', 'paper_title']

db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ap =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['author_names', 'paper_title','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_apy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_appv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['author_names', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_apvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_appvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)



selected_columns = ['paper_title', 'publication_venue']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ppv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)


selected_columns = ['paper_title', 'publication_venue','year']
db = hash.initial_hash(dblp, selected_columns)
am = hash.initial_hash(acm, selected_columns)
similairty_ppvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, selected_indices)

Processing time: 8.483705043792725 seconds. Number of similar pairs: 1235
Processing time: 8.375733137130737 seconds. Number of similar pairs: 1210
Processing time: 8.069044828414917 seconds. Number of similar pairs: 1234
Processing time: 7.951256036758423 seconds. Number of similar pairs: 1536
Processing time: 10.19598388671875 seconds. Number of similar pairs: 1210
Processing time: 9.851144790649414 seconds. Number of similar pairs: 1648
Processing time: 10.36021089553833 seconds. Number of similar pairs: 1554


In [15]:
dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)

dblp['publication_venue'] = dblp['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')
acm['publication_venue'] = acm['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')

threshold = 0.7
indices = ['hash_value']
set_indices = set(indices)

selected_columns = ['author_names', 'paper_title']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ap =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ay =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ay =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'paper_title','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_apy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_appv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['author_names', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_apvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_appvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['paper_title', 'year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_py =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['paper_title', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ppv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['paper_title', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ppvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

Processing time: 9.35474705696106 seconds. Number of similar pairs: 1099
Processing time: 7.363554000854492 seconds. Number of similar pairs: 1485
Processing time: 6.563066005706787 seconds. Number of similar pairs: 2517
Processing time: 8.183785915374756 seconds. Number of similar pairs: 1067
Processing time: 8.3888099193573 seconds. Number of similar pairs: 1096
Processing time: 7.825464963912964 seconds. Number of similar pairs: 1375
Processing time: 8.393160820007324 seconds. Number of similar pairs: 1065
Processing time: 8.818619966506958 seconds. Number of similar pairs: 1518
Processing time: 8.21139907836914 seconds. Number of similar pairs: 1692
Processing time: 9.742849826812744 seconds. Number of similar pairs: 1516


In [16]:
dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)


threshold = 0.85
indices = ['hash_value']
set_indices = set(indices)

selected_columns = ['author_names', 'paper_title']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ap =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ay =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ay =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'paper_title','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_apy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_appv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['author_names', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_apvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['author_names', 'paper_title', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_appvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['paper_title', 'year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_py =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

selected_columns = ['paper_title', 'publication_venue']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ppv =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)


selected_columns = ['paper_title', 'publication_venue','year']
db = hash.hash_blocking(dblp, selected_columns)
am = hash.hash_blocking(acm, selected_columns)
similairty_ppvy =apply_similarity_blocks(db, am, threshold, sim.jaccard_similarity, set_indices)

Processing time: 8.047600030899048 seconds. Number of similar pairs: 1099
Processing time: 7.638789892196655 seconds. Number of similar pairs: 1485
Processing time: 7.817211866378784 seconds. Number of similar pairs: 0
Processing time: 8.465054035186768 seconds. Number of similar pairs: 1067
Processing time: 8.444101810455322 seconds. Number of similar pairs: 0
Processing time: 7.631450891494751 seconds. Number of similar pairs: 0
Processing time: 9.535134077072144 seconds. Number of similar pairs: 0
Processing time: 8.04947805404663 seconds. Number of similar pairs: 1518
Processing time: 8.973160028457642 seconds. Number of similar pairs: 0
Processing time: 8.039626121520996 seconds. Number of similar pairs: 0


In [18]:
threshold = 0.3
n = 2

dblp['year'] = dblp['year'].astype(str)
acm['year'] = acm['year'].astype(str)

selected_columns = ['author_names', 'publication_venue']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_ap = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['author']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_a = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

selected_columns = ['paper_title']
db = ngram.n_gram_blocking(dblp, n, selected_columns)
am = ngram.n_gram_blocking(acm, n, selected_columns)
sim_p = apply_similarity_blocks(db, am, threshold, sim.n_gram_similarity, selected_columns)

Processing time: 76.93934488296509 seconds. Number of similar pairs: 5734
Processing time: 57.52985191345215 seconds. Number of similar pairs: 2580213


In [41]:
import pandas as pd
def block_by_year_ranges(df):
    year_ranges = []
    year_block = [1995, 1998, 2001, 2005]
    labels = ["1995-1997", "1998-2000", "2001-2004"]

    df["year_range"] = pd.cut(df['year'], bins=year_block, labels=labels, right=False)
    for label in labels:
        intervals = df[df["year_range"] == label].to_numpy()
        year_ranges.append(intervals)

    return year_ranges


def block_by_year_ranges_key_dynamic(df, year_block, labels):
    year_ranges = []
    df["year_range"] = pd.cut(df['year'], bins=year_block, labels=labels, right=False)
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[df["year_range"] == label]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                # publisher_block.iterrows() for  all columns?
                publisher_blocks.append(publisher_block)

        year_ranges.extend(publisher_blocks)

    return year_ranges


dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges(dblp)
print(len(dblp_blocks))
acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)
acm_block = block_by_year_ranges(acm)

year_block = [1995, 1998, 2001, 2005]
labels = ["1995-1997", "1998-2000", "2001-2004"]
dblp_blocks = block_by_year_ranges_key_dynamic(dblp, year_block, labels)
length = len(dblp_blocks[0]) + len(dblp_blocks[1]) + len(dblp_blocks[2]) + len(dblp_blocks[3]) + len(dblp_blocks[4]) + len(dblp_blocks[5])
print(length)
print(dblp_blocks[1])

3
2153
                                            paper_title  \
180   constructing efficient decision trees by using...   
183   promises and realities of active database systems   
186   distributed processing over standalone systems...   
189   a region splitting strategy for physical datab...   
190   multidimensional access methods trees have gro...   
...                                                 ...   
2082  the hbpitree a multiattribute index supporting...   
2085  dictionarybased orderpreserving string compres...   
2092  the design and implementation of k a highlevel...   
2093  data model for extensible support of explicit ...   
2094  historical queries along multiple lines of tim...   

                                           author_names  year  \
180   takeshi fukuda, yasuhiko morimoto, shinichi mo...  1996   
183                  eric simon, angelika kotz dittrich  1995   
186   gustavo alonso, claus hagen, hans-jörg schek, ...  1997   
189   jong-hak lee, youn

In [46]:
def block_by_year_ranges_key_dynamic(df, year_block, labels):
    year_ranges = []
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[(df['year'] >= year_block[labels.index(label)]) & (df['year'] < year_block[labels.index(label) + 1])]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if not publisher_block.empty:
                publisher_blocks.append(publisher_block.to_dict(orient='records'))

        year_ranges.extend(publisher_blocks)

    return year_ranges


dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges(dblp)
print(len(dblp_blocks))
acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)
acm_block = block_by_year_ranges(acm)

year_block = [1995, 1998, 2001, 2005]
labels = ["1995-1997", "1998-2000", "2001-2004"]
dblp_blocks = block_by_year_ranges_key_dynamic(dblp, year_block, labels)
length = len(dblp_blocks[0]) + len(dblp_blocks[1]) + len(dblp_blocks[2]) + len(dblp_blocks[3]) + len(dblp_blocks[4]) + len(dblp_blocks[5])
print(length)
print(dblp_blocks[5])

3
2153
[{'paper_title': 'information discovery extraction and integration for the hidden web', 'author_names': 'jiying wang', 'year': 2003, 'publication_venue': 'vldb phd workshop', 'index': '53e9a6f5b7602d9703028288', 'year_range': '2001-2004'}, {'paper_title': 'proceedings of the vldb 2003 phd workshop colocated with the 29th international conference on very large data bases vldb 2003 berlin september 1213 2003', 'author_names': 'unamed', 'year': 2003, 'publication_venue': 'vldb phd workshop', 'index': '53e99d88b7602d9702645354', 'year_range': '2001-2004'}, {'paper_title': 'db2 spatial extender spatial data within the rdbms', 'author_names': 'david w. adler', 'year': 2001, 'publication_venue': 'vldb', 'index': '53e9b1c9b7602d9703c5b2cc', 'year_range': '2001-2004'}, {'paper_title': 'trends in data warehousing a practitioners view', 'author_names': "william o'connell", 'year': 2004, 'publication_venue': 'vldb', 'index': '53e9b08ab7602d9703af6afa', 'year_range': '2001-2004'}, {'paper_ti

In [29]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    for block1 in blocks1:
        for block2 in blocks2:
            for key1, sub_block1 in block1.items():
                for key2, sub_block2 in block2.items():
                    average_similarity = 0.0

                    value_block1 = [sub_block1.get(i, '') for i in indices]
                    value_block2 = [sub_block2.get(i, '') for i in indices]
                    similarity = similarity_function(value_block1, value_block2)

                    average_similarity += similarity

                    if len(indices) > 1:
                        average_similarity /= len(indices)

                    if average_similarity > threshold:
                        index_key1 = sub_block1.get('index', '')
                        index_key2 = sub_block2.get('index', '')
                        
                        if index_key1 and index_key2:
                            index_pairs = [(i, j) for i in sub_block1['index'] for j in sub_block2['index']]
                            similar_pairs.extend(index_pairs)

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs




In [31]:
def block_by_year_ranges_key(df):
    year_ranges = []
    year_block = [1995, 1998, 2001, 2005]
    labels = ["1995-1997", "1998-2000", "2001-2004"]

    df["year_range"] = pd.cut(df['year'], bins=year_block, labels=labels, right=False)
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[df["year_range"] == label]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                publisher_blocks.append(publisher_block)

        year_ranges.extend(publisher_blocks)

    return year_ranges
    

dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key(dblp)
length = len(dblp_blocks[0]) + len(dblp_blocks[1]) + len(dblp_blocks[2]) + len(dblp_blocks[3]) + len(dblp_blocks[4]) + len(dblp_blocks[5])
print(len(dblp_blocks))

6


In [53]:
def block_by_year_ranges_key(df):
    year_ranges = []
    year_block = [1995, 1998, 2001, 2005]
    labels = ["1995-1997", "1998-2000", "2001-2004"]

    df["year_range"] = pd.cut(df['year'], bins=year_block, labels=labels, right=False)
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[df["year_range"] == label]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                publisher_blocks.append(publisher_block.to_dict(orient='records'))

        # Concatenate all publisher blocks into a single list
        year_ranges.extend(publisher_blocks)

    return year_ranges



In [39]:
def block_by_year_ranges_key(df):
    year_ranges = []
    year_block = [1995, 1998, 2001, 2005]
    labels = ["1995-1997", "1998-2000", "2001-2004"]

    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[df["year"].between(year_block[0], year_block[-1], inclusive='left')]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                publisher_blocks.extend(publisher_block.to_dict(orient='records'))

        # Use the label as the key for each publisher block
        year_ranges.append(publisher_blocks)

    return year_ranges

dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key(dblp)
print(len(dblp_blocks))


3


In [33]:
def block_by_year_ranges_key(df):
    year_ranges = []
    year_block = [1995, 1998, 2001, 2005]

    publishers = ['sigmod','vldb']  # List of publishers to separate data based on

    for year in range(year_block[0], year_block[-1]):
        publisher_blocks = []
        intervals = df[df["year"] == year]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                publisher_blocks.extend(publisher_block.to_dict(orient='records'))

        # Append the publisher blocks for the current year
        year_ranges.append(publisher_blocks)

    return year_ranges



In [37]:
dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key(dblp)
print(dblp_blocks[7])

[{'paper_title': 'editors notes', 'author_names': 'ling liu', 'year': 2002, 'publication_venue': 'sigmod record', 'index': '53e99800b7602d970200b603'}, {'paper_title': 'tods perceptions and misconceptions', 'author_names': 'richard t. snodgrass', 'year': 2002, 'publication_venue': 'sigmod record', 'index': '53e99ae6b7602d970236d5f2'}, {'paper_title': 'sqlmed a status report', 'author_names': 'jim melton, jan-eike michels, vanja josifovski, krishna g. kulkarni, peter m. schwarz', 'year': 2002, 'publication_venue': 'sigmod record', 'index': '53e9b4d4b7602d9703ff2db2'}, {'paper_title': 'a multiagent system infrastructure for software component marketplace an ontological perspective', 'author_names': 'riza cenk erdur, oguz dikenelli', 'year': 2002, 'publication_venue': 'sigmod record', 'index': '53e9bd8cb7602d9704a359db'}, {'paper_title': 'semantic b2b integration issues in ontologybased applications', 'author_names': "zhan cui, dean m. jones, paul o'brien", 'year': 2002, 'publication_venu

In [12]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    for sub_block1, sub_block2 in zip(blocks1, blocks2):
        average_similarity = 0.0

        value_block1 = [item.get(i, '') for item in sub_block1 for i in indices]
        value_block2 = [item.get(i, '') for item in sub_block2 for i in indices]
        similarity = similarity_function(value_block1, value_block2)

        average_similarity += similarity

        if len(indices) > 1:
            average_similarity /= len(indices)

        if average_similarity >= threshold:
            index_key1 = [item.get('index', '') for item in sub_block1]
            index_key2 = [item.get('index', '') for item in sub_block2]

            if index_key1 and index_key2:
                index_pairs = [(i, j) for i in index_key1 for j in index_key2]
                similar_pairs.extend(index_pairs)

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs





In [24]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    # Compare each block in blocks1 with each block in blocks2
    for block1 in blocks1:
        for block2 in blocks2:
            average_similarity = 0.0

            value_block1 = [str(block1).lower()]
            value_block2 = [str(block2).lower()]

            # Calculate similarity
            similarity = similarity_function(value_block1, value_block2)
            average_similarity += similarity
            
            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                similar_pairs.append((block1, block2, average_similarity))

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    #print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs




In [30]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    # Compare each block in blocks1 with each block in blocks2
    for subblock1, subblock2 in zip(blocks1, blocks2):
            average_similarity = 0.0
            value_block1 = [item.get(i, '') for item in subblock1 for i in indices]
            value_block2 = [item.get(i, '') for item in subblock2 for i in indices]

            # Calculate similarity
            similarity = similarity_function(value_block1, value_block2)
            average_similarity += similarity
            
            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                similar_pairs.append((subblock1[index], subblock2[index], average_similarity))

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    #print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs


In [ ]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    # Record the start time
    start_time = time.time()

    # Compare each block in blocks1 with each block in blocks2
    for block1 in blocks1:
        for block2 in blocks2:
            average_similarity = 0.0

            value_block1 = [str(block1.get(index, '')).lower() for index in indices]
            value_block2 = [str(block2.get(index, '')).lower() for index in indices]

            # Calculate similarity
            similarity = similarity_function(value_block1, value_block2)
            average_similarity += similarity
            
            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                similar_pairs.append((block1, block2, average_similarity))

    # Record the end time
    end_time = time.time()

    # Calculate the elapsed time
    elapsed_time = end_time - start_time

    # Print the elapsed time
    #print(f"Processing time: {elapsed_time} seconds. Number of similar pairs: {len(similar_pairs)}")

    return similar_pairs

In [77]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    for block1 in blocks1:
        for block2 in blocks2:
            average_similarity = 0.0

            for index in indices:
                value_block1 = block1.get(index, '')
                value_block2 = block2.get(index, '')

                similarity = similarity_function(value_block1, value_block2)
                average_similarity += similarity

            if len(indices) > 0:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                index_pairs = [(i, j) for i in block1['index'] for j in block2['index']]
                similar_pairs.extend(index_pairs)

    return similar_pairs


In [80]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    for block1, block2 in zip(blocks1, blocks2):
            average_similarity = 0.0

            for index in indices:
                value_block1 = block1.get(index, '')
                value_block2 = block2.get(index, '')

                similarity = similarity_function(value_block1, value_block2)
                average_similarity += similarity

            if len(indices) > 0:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                index_pairs = [(i, j) for i in block1['index'] for j in block2['index']]
                similar_pairs.extend(index_pairs)

    return similar_pairs

In [71]:
def block_by_year_ranges_key_dynamic(df, year_block, labels):
    year_ranges = []
    df["year_range"] = pd.cut(df['year'], bins=year_block, labels=labels, right=False)
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[df["year_range"] == label]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if publisher_block.size > 0:
                # publisher_block.iterrows() for  all columns?
                publisher_blocks.append(publisher_block)

        year_ranges.extend(publisher_blocks)

    return year_ranges

In [81]:
def block_by_year_ranges_key_dynamic(df, year_block, labels):
    year_ranges = []
    publishers = ['sigmod', 'vldb']  # List of publishers to separate data based on

    for label in labels:
        publisher_blocks = []
        intervals = df[(df['year'] >= year_block[labels.index(label)]) & (df['year'] < year_block[labels.index(label) + 1])]

        for publisher in publishers:
            publisher_block = intervals[intervals["publication_venue"].str.contains(publisher)]
            if not publisher_block.empty:
                publisher_blocks.append(publisher_block.to_dict(orient='records'))

        year_ranges.extend(publisher_blocks)

    return year_ranges


year_block = [1995,1996,1997, 1998, 1999,2000,2001, 2002, 2003, 2004,2005]
labels = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004"]


In [59]:
dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'

dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key_dynamic(dblp, year_block, labels)
print((dblp_blocks[3]))
#acm = pd.read_csv(acm_csv)
#acm_blocks = block_by_year_ranges_key_dynamic(acm)

[{'paper_title': 'constructing efficient decision trees by using optimized numeric association rules', 'author_names': 'takeshi fukuda, yasuhiko morimoto, shinichi morishita, takeshi tokuyama', 'year': 1996, 'publication_venue': 'vldb', 'index': '53e9b5b6b7602d97040fa1e4'}, {'paper_title': 'extracting large data sets using db2 parallel edition', 'author_names': 'sriram padmanabhan', 'year': 1996, 'publication_venue': 'vldb', 'index': '53e99b0ab7602d970239bff2'}, {'paper_title': 'eroc a toolkit for building neato query optimizers', 'author_names': 'william j. mckenna, louis burger, chi hoang, melissa truong', 'year': 1996, 'publication_venue': 'vldb', 'index': '53e9affbb7602d9703a55249'}, {'paper_title': 'scalablity and availability in oracle7 73', 'author_names': 'dieter gawlick', 'year': 1996, 'publication_venue': 'vldb', 'index': '53e99bcdb7602d9702479cdc'}, {'paper_title': 'applying data mining techniques to a health insurance information system', 'author_names': 'marisa s. viveros,

In [127]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []
    for block1 in blocks1:
        for block2 in blocks2:
            average_similarity = 0.0

            for index in indices:
                value_block1 = [elem.get(index, '') for elem in block1]
                value_block2 = [elem.get(index, '') for elem in block2]

                similarity = similarity_function(value_block1, value_block2)
                average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                index_pairs = [(elem1['index'], elem2['index']) for elem1 in block1 for elem2 in block2]
                similar_pairs.extend(index_pairs)

    return similar_pairs


In [131]:
import pandas as pd

dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'

year_block = [1995,1996,1997, 1998, 1999,2000,2001, 2002, 2003, 2004,2005]
labels = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004"]

dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key_dynamic(dblp, year_block, labels)

acm = pd.read_csv(acm_csv)
acm_blocks = block_by_year_ranges_key_dynamic(acm, year_block, labels)

indices = ['author_names','paper_title']
set_indices = set(indices) 
threshold = 0.7
total = 0

match = apply_similarity_blocks(dblp_blocks, acm_blocks, threshold, sim.jaccard_similarity, set_indices)
print(len(match))

0


In [129]:
def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    for block1, block2 in zip(blocks1, blocks2):
        average_similarity = 0.0

        for index in indices:
            value_block1 = [elem.get(index, '') for elem in block1]
            value_block2 = [elem.get(index, '') for elem in block2]

            similarity = similarity_function(value_block1, value_block2)
            average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity >= threshold:
                index_pairs = [(elem1['index'], elem2['index']) for elem1 in block1 for elem2 in block2]
                similar_pairs.extend(index_pairs)

    return similar_pairs


In [133]:
import pandas as pd

dblp_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/dblp.csv'
acm_csv = '/Users/aliaslan/Downloads/DIA_Exercise/CSV-files/acm.csv'

year_block = [1995,1996,1997, 1998, 1999,2000,2001, 2002, 2003, 2004,2005]
labels = ["1995", "1996", "1997", "1998", "1999", "2000", "2001", "2002", "2003", "2004"]

dblp = pd.read_csv(dblp_csv)
dblp_blocks = block_by_year_ranges_key_dynamic(dblp, year_block, labels)

acm = pd.read_csv(acm_csv)
acm_blocks = block_by_year_ranges_key_dynamic(acm, year_block, labels)

indices = ['author_names','paper_title']
set_indices = set(indices) 
threshold = 0.6
total = 0

match = apply_similarity_blocks(dblp_blocks, acm_blocks, threshold, sim.jaccard_similarity, set_indices)
print(len(match))


6776


Blocking with Hash - choose columns freely

In [7]:
import hashlib

def hash_blocking(dataframe, blocking_columns):
    dataframe['hash_key'] = dataframe[blocking_columns].astype(str).agg(''.join, axis=1)
    dataframe['hash_value'] = dataframe['hash_key'].apply(lambda x: hashlib.md5(x.encode()).hexdigest())
    blocks = dataframe.groupby('hash_value').apply(lambda x: {'hash_value': x['hash_value'].iloc[0], 'index': x['index'].tolist()}).to_dict()
    return blocks

Blocking by initials combined and the hashed

In [6]:
import hashlib
import pandas as pd



def initial_hash(df, columns):
    df ['publication_venue'] = df['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')
    blocks = {}
    for index, row in df.iterrows():
        components_values = []

        for column in columns:
            if column == 'author_names':
                author_initials = []
                for name in row['author_names'].split():
                    parts = name.split()
                    if len(parts) > 1:
                        author_initials.append(parts[0][0] + parts[-1][0])
                    else:
                        author_initials.append(name[0])
                component_value = "".join(author_initials)
            elif column == 'paper_title':
                component_value = "".join(word[0] for word in row['paper_title'].split())
            elif column == 'publication_venue':
                component_value = row['publication_venue']
            elif column == 'year':
                component_value = str(row["year"])
            

            components_values.append(component_value)

        blocking_key = ''.join(components_values)
        blocking_key_hash = hashlib.md5(blocking_key.encode()).hexdigest()
        block_data = {'ngram_values': blocking_key_hash, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

Blocking with hash

N-Gram blocking - wähl die Columns freiwillig aus 

In [ ]:
import hashlib

def ngrams_tuple(s, n):
    return [tuple(s[i:i+n]) for i in range(len(s)-n+1)]

def hash_ngram_values(ngram_values):
    return hashlib.sha256(str(hash(tuple(sorted(ngram_values.items())))).encode()).hexdigest()

def n_gram_blocking(dataframe, columns, n):
    # n-gram for each selected column 
    for column in columns:
        key_name = f'ngram_key_{column}'
        dataframe[key_name] = dataframe[column].apply(lambda x: ''.join(map(str, x)))

        ngram_col_name = f'ngram_values_{column}'
        dataframe[ngram_col_name] = dataframe[key_name].apply(lambda x: tuple(ngrams_tuple(x, n)))

    # block them and each block consists of n-gram values and the respective index (id) 
    blocks = {}
    for index, row in dataframe.iterrows():
        ngram_values = {column: row[f'ngram_values_{column}'] for column in columns}
        hashable_ngram_values = hashlib.sha256(str(hash(tuple(sorted(ngram_values.items())))).encode()).hexdigest()
        
        if hashable_ngram_values in blocks:
            blocks[hashable_ngram_values]['index'].add(row['index'])
        else:
            blocks[hashable_ngram_values] = {'index': {row['index']}, **ngram_values}

    return blocks

# if you want to try out
""""
selected_columns = ['author_names', 'paper_title']
n_value = 2
result_blocks = n_gram_blocking(dblp, selected_columns, n_value)

for key, block in result_blocks.items():
    print(f"Block key: {key}")
    print(f"Block content: {block}")
    print(f"Values for selected columns: {[block.get(i, '') for i in selected_columns]}")
    print("-----")
"""

Initials als combined string mit n-gram wobei Jahr komplett genommen wird und beim publisher_venue nur das Stichwort sigmod or vldb

In [32]:


def ngrams_string(s, n):
    return [s[i:i+n] for i in range(len(s)-n+1)]

# created this afterwards so we dont need to use the other initial_..._ngram methods (didnt delete the others for know, 
# because I used them for matching can be changed after we decide our baseline)
def initial_ngram(df, n, columns):
    df ['publication_venue'] = df['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')
    blocks = {}
    for index, row in df.iterrows():
        components_values = []

        for column in columns:
            if column == 'author_names':
                author_initials = []
                for name in row['author_names'].split():
                    parts = name.split()
                    if len(parts) > 1:
                        author_initials.append(parts[0][0] + parts[-1][0])
                    else:
                        author_initials.append(name[0])
                component_value = "".join(author_initials)
            elif column == 'paper_title':
                component_value = "".join(word[0] for word in row['paper_title'].split())
            elif column == 'publication_venue':
                component_value = row['publication_venue']
            elif column == 'year':
                component_value = str(row["year"])
        
            components_values.append(component_value)

        blocking_key = ''.join(components_values)
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

In [12]:
import pandas as pd

def ngrams_string(s, n):
    return [s[i:i+n] for i in range(len(s)-n+1)]

# ap = author + paper title

def initial_ap_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

        combined_author_initials = "".join(author_initials)
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())

        blocking_key = combined_author_initials + paper_title_initials
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

# author + publication venue
def initial_apv_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

        combined_author_initials = "".join(author_initials)

        venue_key = row["publication_venue"] 

        publisher_keyword = "sigmod" if "sigmod" in venue_key else "vldb"
        blocking_key = combined_author_initials + publisher_keyword
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks


# author + year
def initial_ay_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

        combined_author_initials = "".join(author_initials)
        
        year_key = str(row["year"])
    
        blocking_key = combined_author_initials + year_key
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

def initial_apy_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

        combined_author_initials = "".join(author_initials)
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())
        year_key = str(row["year"])

        blocking_key = combined_author_initials + paper_title_initials + year_key
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

   

def initial_apvy_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

    
        combined_author_initials = "".join(author_initials)
        year_key = str(row["year"])
        venue_key = row["publication_venue"] 

     
        publisher_keyword = "sigmod" if "sigmod" in venue_key else "vldb"
        blocking_key = combined_author_initials + publisher_keyword + year_key
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

def initial_appvy_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        author_initials = []
        for name in row['author_names'].split():
            parts = name.split()
            if len(parts) > 1:
                author_initials.append(parts[0][0] + parts[-1][0])
            else:
                author_initials.append(name[0])

         
        combined_author_initials = "".join(author_initials)
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())

        year_key = str(row["year"])
        venue_key = row["publication_venue"] 

        # Directly use "sigmod" or "vldb" in the blocking key
        publisher_keyword = "sigmod" if "sigmod" in venue_key else "vldb"
        blocking_key =  combined_author_initials + paper_title_initials + year_key  + publisher_keyword
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

def initial_py_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())
        year_key = str(row["year"])

        blocking_key = paper_title_initials + year_key
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

# pp = paper_tite and publication venue
def initial_ppv_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())
               
        venue_key = row["publication_venue"] 

        # Directly use "sigmod" or "vldb" in the blocking key
        publisher_keyword = "sigmod" if "sigmod" in venue_key else "vldb"
        blocking_key = paper_title_initials + publisher_keyword
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks

# like method before but with year
def initial_ppvy_ngram(df, n):
    blocks = {}
    for index, row in df.iterrows():
        paper_title_initials = "".join(word[0] for word in row['paper_title'].split())
               
        year_key = str(row["year"])
        venue_key = row["publication_venue"] 

        # Directly use "sigmod" or "vldb" in the blocking key
        publisher_keyword = "sigmod" if "sigmod" in venue_key else "vldb"
        blocking_key = paper_title_initials + year_key  + publisher_keyword
        blocking_key_ngrams = list(ngrams_string(blocking_key, n))
        block_data = {'ngram_values': blocking_key_ngrams, 'index': [row['index']]}

        if blocking_key in blocks:
            unique_indices = list(set(blocks[blocking_key]['index'] + block_data['index']))
            blocks[blocking_key]['index'] = unique_indices
        else:
            blocks[blocking_key] = block_data

    return blocks


Similarity methods

In [36]:

def apply_similarity(df1, df2, threshold, selected_columns, similarity_function):
    similar_pairs = []

    for index1, row1 in df1.iterrows():
        for index2, row2 in df2.iterrows():
            average_similarity = 0.0

            for col_df1, col_df2 in zip(selected_columns, selected_columns):
                value_df1 = set(row1[col_df1].lower().split())
                value_df2 = set(row2[col_df2].lower().split())

                similarity = similarity_function(value_df1, value_df2)
                average_similarity += similarity

            if len(selected_columns) > 1:
                average_similarity /= len(selected_columns)

            if average_similarity >= threshold:
                pair = (row1[selected_columns].to_dict(), row2[selected_columns].to_dict(), average_similarity)
                similar_pairs.append(pair)

    return similar_pairs



def apply_similarity_blocks(blocks1, blocks2, threshold, similarity_function, indices):
    similar_pairs = []

    for key1, block1 in blocks1.items():
        for key2, block2 in blocks2.items():
            average_similarity = 0.0

            value_block1 = [block1.get(i, '') for i in indices]
            value_block2 = [block2.get(i, '') for i in indices]
            similarity = similarity_function(value_block1, value_block2)

            average_similarity += similarity

            if len(indices) > 1:
                average_similarity /= len(indices)

            if average_similarity > threshold:
                index_pairs = [(i, j) for i in block1['index'] for j in block2['index']]
                similar_pairs.extend(index_pairs)
                
                
    return similar_pairs



In [8]:
# get intersection and union then divide it
#def jaccard_similarity(set1, set2):
#    intersection = len(set1 & set2)
#    union = len(set1 | set2)
#    return intersection / union if union > 0 else 0


def jaccard_similarity(set1, set2):
    if not isinstance(set1, set):
        set1 = set(set1)
    if not isinstance(set2, set):
        set2 = set(set2)
    
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    
    return intersection / union if union > 0 else 0

In [34]:
import numpy as np
def levensthein_distance(set1, set2):
    # make the subcriptable 
    list1, list2 = list(set1), list(set2)
    m, n = len(list1), len(list2)

    # create matrix
    matrix = np.zeros((m + 1, n + 1))

    # fill row with length of string 
    for i in range(1, m + 1):
        matrix[i][0] = i

    # fill column with lenth of string 
    for j in range(1, n + 1):
        matrix[0][j] = j
    
    for i in range(1, m + 1):
        for j in range(1, n + 1):
            if list1[i - 1] ==  list2[j - 1]:
                cost = 0
            else:
                cost = 1
            
            matrix[i][j] = min(matrix[i - 1][j] + 1, matrix[i][j - 1] + 1, matrix[i - 1][j - 1] + cost)
    similarity = 1 - (matrix[m][n] / (m + n))

    return similarity

In [33]:
# get n_gram
def get_n_grams(string, n):
    characters = list(string.lower())
    n_grams = [tuple(characters[i:i+n]) for i in range(len(characters) - (n-1))]
    return n_grams

# transform respective columns to n_gram
def transform_columns_to_n_gram(df, n, columns):
    for column in columns:
        df[f'{column}'] = df[column].apply(lambda x: get_n_grams(str(x), n))
    return df


# as in lecture
"""def n_gram_similarity(df1, df2):
    df1, df2 = set(df1), set(df2)
    intersection = len(df1) & len(df2)
    return 2 * intersection / len(df1) + len(df2)"""

def n_gram_similarity(df1, df2):
    set_df1 = {item for sublist in df1 for item in sublist}
    set_df2 = {item for sublist in df2 for item in sublist}
    intersection = len(set_df1 & set_df2)
    return 2 * intersection / (len(set_df1) + len(set_df2))

Matching of all initial with n_gram with two different n and treshholds!

In [17]:
import csv

def similar_pairs_to_csv(similar_pairs, output_csv_file):
    header = ['dblp_index', 'acm_index']
    with open(output_csv_file, mode='w', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        for pair in similar_pairs:
            writer.writerow(pair)


In [9]:
dblp_csv = '../CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = '../CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)

threshold = 0.7

selected_indices = ['ngram_values']
selected_columns = ['author_names', 'paper_title']

db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_ap =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_ap))

selected_columns = ['author_names', 'paper_title','year']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_apy =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_apy))

selected_columns = ['author_names', 'paper_title', 'publication_venue']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_appv =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_appv))

selected_columns = ['author_names', 'publication_venue','year']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_apvy =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_apvy))

selected_columns = ['author_names', 'paper_title', 'publication_venue','year']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_appvy =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_appvy))


selected_columns = ['paper_title', 'publication_venue']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_ppv =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_appv))

selected_columns = ['paper_title', 'publication_venue','year']
db = initial_hash(dblp, selected_columns)
am = initial_hash(acm, selected_columns)
similairty_ppvy =apply_similarity_blocks(db, am, threshold, jaccard_similarity, selected_indices)
print(len(similairty_ppvy))


1235
1210
1234
1536
1210
1234
1554


In [46]:
dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)


threshold = 0.7
n = 2
selected_indices = ['ngram_values']

dblp_ap = initial_ap_ngram(dblp, n)
acm_ap = initial_ap_ngram(acm, n)
sim_ap = apply_similarity_blocks(dblp_ap, acm_ap, threshold, n_gram_similarity, selected_indices)
print(len(sim_ap))
similar_pairs_to_csv(sim_ap, 'similarity_th_07/sim_ap.csv')

dblp_apv = initial_apv_ngram(dblp, n)
acm_apv = initial_apv_ngram(acm, n)
sim_apv = apply_similarity_blocks(dblp_apv, acm_apv, threshold, n_gram_similarity, selected_indices)
print(len(sim_apv))
similar_pairs_to_csv(sim_apv, 'similarity_th_07/sim_apv.csv')

dblp_ay = initial_ay_ngram(dblp, n)
acm_ay = initial_ay_ngram(acm, n)
sim_ay = apply_similarity_blocks(dblp_ay, acm_ay, threshold, n_gram_similarity, selected_indices)
print(len(sim_ay))
similar_pairs_to_csv(sim_ay, 'similarity_th_07/sim_ay.csv')

dblp_apy = initial_apy_ngram(dblp, n)
acm_apy = initial_apy_ngram(acm, n)
sim_apy = apply_similarity_blocks(dblp_apy, acm_apy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apy))
similar_pairs_to_csv(sim_apy, 'similarity_th_07/sim_apy.csv')


dblp_apvy = initial_apvy_ngram(dblp, n)
acm_apvy = initial_apvy_ngram(acm, n)
sim_apvy = apply_similarity_blocks(dblp_apvy, acm_apvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apvy))
similar_pairs_to_csv(sim_apvy, 'similarity_th_07/sim_apvy.csv')

dblp_appvy = initial_appvy_ngram(dblp, n)
acm_appvy = initial_appvy_ngram(acm, n)
sim_appvy = apply_similarity_blocks(dblp_appvy, acm_appvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_appvy))
similar_pairs_to_csv(sim_appvy, 'similarity_th_07/sim_appvy.csv')

dblp_py = initial_py_ngram(dblp, n)
acm_py = initial_py_ngram(acm, n)
sim_py = apply_similarity_blocks(dblp_py, acm_py, threshold, n_gram_similarity, selected_indices)
print(len(sim_py))
similar_pairs_to_csv(sim_py, 'similarity_th_07/sim_py.csv')

dblp_ppv = initial_ppv_ngram(dblp, n)
acm_ppv = initial_ppv_ngram(acm, n)
sim_ppv = apply_similarity_blocks(dblp_ppv, acm_ppv, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppv))
similar_pairs_to_csv(sim_ppv, 'similarity_th_07/sim_ppv.csv')

dblp_ppvy = initial_ppvy_ngram(dblp, n)
acm_ppvy = initial_ppvy_ngram(acm, n)
sim_ppvy = apply_similarity_blocks(dblp_ppvy, acm_ppvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppvy))
similar_pairs_to_csv(sim_ppvy, 'similarity_th_07/sim_ppvy.csv')

1775
121667
6195
1801
128842
2016
2243
11216
11701


In [ ]:
threshold = 0.9
n = 2
selected_indices = ['ngram_values']

dblp_ap = initial_ap_ngram(dblp, n)
acm_ap = initial_ap_ngram(acm, n)
sim_ap2 = apply_similarity_blocks(dblp_ap, acm_ap, threshold, n_gram_similarity, selected_indices)
print(len(sim_ap2))
similar_pairs_to_csv(sim_ap2, 'similarity_th_09/sim_ap.csv')

dblp_apv = initial_apv_ngram(dblp, n)
acm_apv = initial_apv_ngram(acm, n)
sim_apv2 = apply_similarity_blocks(dblp_apv, acm_apv, threshold, n_gram_similarity, selected_indices)
print(len(sim_apv2))
similar_pairs_to_csv(sim_apv2, 'similarity_th_09/sim_apv.csv')

dblp_ay = initial_ay_ngram(dblp, n)
acm_ay = initial_ay_ngram(acm, n)
sim_ay2 = apply_similarity_blocks(dblp_ay, acm_ay, threshold, n_gram_similarity, selected_indices)
print(len(sim_ay2))
similar_pairs_to_csv(sim_ay2, 'similarity_th_09/sim_ay.csv')

dblp_apy = initial_apy_ngram(dblp, n)
acm_apy = initial_apy_ngram(acm, n)
sim_apy2 = apply_similarity_blocks(dblp_apy, acm_apy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apy2))
similar_pairs_to_csv(sim_apy2, 'similarity_th_09/sim_apy.csv')

dblp_apvy = initial_apvy_ngram(dblp, n)
acm_apvy = initial_apvy_ngram(acm, n)
sim_apvy2 = apply_similarity_blocks(dblp_apvy, acm_apvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apvy2))
similar_pairs_to_csv(sim_apvy2, 'similarity_th_09/sim_apvy.csv')

dblp_appvy = initial_appvy_ngram(dblp, n)
acm_appvy = initial_appvy_ngram(acm, n)
sim_appvy2 = apply_similarity_blocks(dblp_appvy, acm_appvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_appvy2))
similar_pairs_to_csv(sim_appvy2, 'similarity_th_09/sim_appvy.csv')

dblp_py = initial_py_ngram(dblp, n)
acm_py = initial_py_ngram(acm, n)
sim_py2 = apply_similarity_blocks(dblp_py, acm_py, threshold, n_gram_similarity, selected_indices)
print(len(sim_py2))
similar_pairs_to_csv(sim_py2, 'similarity_th_09/sim_py.csv')

dblp_ppv = initial_ppv_ngram(dblp, n)
acm_ppv = initial_ppv_ngram(acm, n)
sim_ppv2 = apply_similarity_blocks(dblp_ppv, acm_ppv, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppv2))
similar_pairs_to_csv(sim_ppv2, 'similarity_th_09/sim_ppv.csv')

dblp_ppvy = initial_ppvy_ngram(dblp, n)
acm_ppvy = initial_ppvy_ngram(acm, n)
sim_ppvy2 = apply_similarity_blocks(dblp_ppvy, acm_ppvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppvy2))
similar_pairs_to_csv(sim_ppvy2, 'similarity_th_09/sim_ppvy.csv')

1405
3505
1895
1456
2896
1540
1637
1807
1664


In [1]:
threshold = 0.7
n = 3
selected_indices = ['ngram_values']

dblp_ap = initial_ap_ngram(dblp, n)
acm_ap = initial_ap_ngram(acm, n)
sim_ap3 = apply_similarity_blocks(dblp_ap, acm_ap, threshold, n_gram_similarity, selected_indices)
print(len(sim_ap3))
similar_pairs_to_csv(sim_ap3, 'similarity_th_07_03/sim_ap.csv')

dblp_apv = initial_apv_ngram(dblp, n)
acm_apv = initial_apv_ngram(acm, n)
sim_apv3 = apply_similarity_blocks(dblp_apv, acm_apv, threshold, n_gram_similarity, selected_indices)
print(len(sim_apv3))
similar_pairs_to_csv(sim_apv3, 'similarity_th_07_03/sim_apv.csv')

dblp_ay = initial_ay_ngram(dblp, n)
acm_ay = initial_ay_ngram(acm, n)
sim_ay3 = apply_similarity_blocks(dblp_ay, acm_ay, threshold, n_gram_similarity, selected_indices)
print(len(sim_ay3))
similar_pairs_to_csv(sim_ay3, 'similarity_th_07_03/sim_ay.csv')

dblp_apy = initial_apy_ngram(dblp, n)
acm_apy = initial_apy_ngram(acm, n)
sim_apy3 = apply_similarity_blocks(dblp_apy, acm_apy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apy3))
similar_pairs_to_csv(sim_apy3, 'similarity_th_07_03/sim_apy.csv')

dblp_apvy = initial_apvy_ngram(dblp, n)
acm_apvy = initial_apvy_ngram(acm, n)
sim_apvy3 = apply_similarity_blocks(dblp_apvy, acm_apvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apvy3))
similar_pairs_to_csv(sim_apvy3, 'similarity_th_07_03/sim_apvy.csv')

dblp_appvy = initial_appvy_ngram(dblp, n)
acm_appvy = initial_appvy_ngram(acm, n)
sim_appvy3 = apply_similarity_blocks(dblp_appvy, acm_appvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_appvy3))
similar_pairs_to_csv(sim_appvy3, 'similarity_th_07_03/sim_appvy.csv')

dblp_py = initial_py_ngram(dblp, n)
acm_py = initial_py_ngram(acm, n)
sim_py3 = apply_similarity_blocks(dblp_py, acm_py, threshold, n_gram_similarity, selected_indices)
print(len(sim_py3))
similar_pairs_to_csv(sim_py3, 'similarity_th_07_03/sim_py.csv')

dblp_ppv = initial_ppv_ngram(dblp, n)
acm_ppv = initial_ppv_ngram(acm, n)
sim_ppv3 = apply_similarity_blocks(dblp_ppv, acm_ppv, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppv3))
similar_pairs_to_csv(sim_ppv3, 'similarity_th_07_03/sim_ppv.csv')

dblp_ppvy = initial_ppvy_ngram(dblp, n)
acm_ppvy = initial_ppvy_ngram(acm, n)
sim_ppvy3 = apply_similarity_blocks(dblp_ppvy, acm_ppvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppvy3))
similar_pairs_to_csv(sim_ppvy3, 'similarity_th_07_03/sim_ppvy.csv')

NameError: name 'initial_ap_ngram' is not defined

In [ ]:
threshold = 0.9
n = 3
selected_indices = ['ngram_values']

dblp_ap = initial_ap_ngram(dblp, n)
acm_ap = initial_ap_ngram(acm, n)
sim_ap4 = apply_similarity_blocks(dblp_ap, acm_ap, threshold, n_gram_similarity, selected_indices)
print(len(sim_ap4))
similar_pairs_to_csv(sim_ap4, 'similarity_th_09_03/sim_ap.csv')

dblp_apv = initial_apv_ngram(dblp, n)
acm_apv = initial_apv_ngram(acm, n)
sim_apv4 = apply_similarity_blocks(dblp_apv, acm_apv, threshold, n_gram_similarity, selected_indices)
print(len(sim_apv4))
similar_pairs_to_csv(sim_apv4, 'similarity_th_09_03/sim_apv.csv')

dblp_ay = initial_ay_ngram(dblp, n)
acm_ay = initial_ay_ngram(acm, n)
sim_ay4 = apply_similarity_blocks(dblp_ay, acm_ay, threshold, n_gram_similarity, selected_indices)
print(len(sim_ay4))
similar_pairs_to_csv(sim_ay4, 'similarity_th_09_03/sim_ay.csv')

dblp_apy = initial_apy_ngram(dblp, n)
acm_apy = initial_apy_ngram(acm, n)
sim_apy4 = apply_similarity_blocks(dblp_apy, acm_apy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apy4))
similar_pairs_to_csv(sim_apy4, 'similarity_th_09_03/sim_apy.csv')

dblp_apvy = initial_apvy_ngram(dblp, n)
acm_apvy = initial_apvy_ngram(acm, n)
sim_apvy4 = apply_similarity_blocks(dblp_apvy, acm_apvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_apvy4))
similar_pairs_to_csv(sim_apvy4, 'similarity_th_09_03/sim_apvy.csv')

dblp_appvy = initial_appvy_ngram(dblp, n)
acm_appvy = initial_appvy_ngram(acm, n)
sim_appvy4 = apply_similarity_blocks(dblp_appvy, acm_appvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_appvy4))
similar_pairs_to_csv(sim_appvy4, 'similarity_th_09_03/sim_appvy.csv')

dblp_py = initial_py_ngram(dblp, n)
acm_py = initial_py_ngram(acm, n)
sim_py4 = apply_similarity_blocks(dblp_py, acm_py, threshold, n_gram_similarity, selected_indices)
print(len(sim_py4))
similar_pairs_to_csv(sim_py4, 'similarity_th_09_03/sim_py.csv')

dblp_ppv = initial_ppv_ngram(dblp, n)
acm_ppv = initial_ppv_ngram(acm, n)
sim_ppv4 = apply_similarity_blocks(dblp_ppv, acm_ppv, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppv4))
similar_pairs_to_csv(sim_ppv4, 'similarity_th_09_03/sim_ppv.csv')

dblp_ppvy = initial_ppvy_ngram(dblp, n)
acm_ppvy = initial_ppvy_ngram(acm, n)
sim_ppvy4 = apply_similarity_blocks(dblp_ppvy, acm_ppvy, threshold, n_gram_similarity, selected_indices)
print(len(sim_ppvy4))
similar_pairs_to_csv(sim_ppvy4, 'similarity_th_09_03/sim_ppvy.csv')

1291
3312
1685
1279
1873
1345
1592
1754
1598


Bemerkung: Wir haben einige Outliner mit Matches mit einer Anzahl von ca 30k-120k Paaren haben. Die Ursache hierbei liegt beim Prozess und die Werte: Beispielsweise nehmen wir bei den initialen Methoden das ganze Jahr und für publication venue extrahieren wir sigmod oder vdlb. Da es beim venue nur die zwei Möglichkeiten gibt kriegen wir für die Ähnlichkeit entweder 1 (identisch) oder x und beim Jahr haben wir 1995-2004 bedeutet bei identischen 1, bei ähnlichen 1995-1999 oder 2000-2004 kriegt man ein hohen Wert.
Natürlich werden die Werte nicht einzeln, sondern als kombinierter String verglichen, aber wenn man nicht genügend andere Werte oder lange = (Anzahl an Characters) Werte hat wie paper_title, dann wird es eben als sehr ähnlich gewertet.

Matching with hashblock

In [10]:
dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)

# make it so it either sigmod or vdlb (could have been done for initial_ngrams too but to lazy to change now)
dblp['publication_venue'] = dblp['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')
acm['publication_venue'] = acm['publication_venue'].apply(lambda x: 'sigmod' if 'sigmod' in x else 'vldb')

threshold = 0.7
indices = ['hash_value']
set_indices = set(indices)

blocking_columns = ['author_names','paper_title']
dblp_hap = hash.hash_blocking(dblp, blocking_columns)
acm_hap = hash.hash_blocking(acm,blocking_columns)
sim_hap = apply_similarity_blocks(dblp_hap, acm_hap, threshold, sim.jaccard_similarity, set_indices)
print(len(sim_hap))


blocking_columns = ['author_names','publication_venue']
dblp_hapv = hash_blocking(dblp, blocking_columns)
acm_hapv = hash_blocking(acm,blocking_columns)
sim_hapv = apply_similarity_blocks(dblp_hapv, acm_hapv, threshold, jaccard_similarity, set_indices)
print(len(sim_hapv))


blocking_columns = ['author_names','year']
dblp_hay = hash_blocking(dblp, blocking_columns)
acm_hay = hash_blocking(acm,blocking_columns)
sim_hay = apply_similarity_blocks(dblp_hay, acm_hay, threshold, jaccard_similarity, set_indices)
print(len(sim_hay))


blocking_columns = ['author_names','paper_title', 'publication_venue']
dblp_happv = hash_blocking(dblp, blocking_columns)
acm_happv = hash_blocking(acm,blocking_columns)
sim_happv = apply_similarity_blocks(dblp_happv, acm_happv, threshold, jaccard_similarity, set_indices)
print(len(sim_happv))


blocking_columns = ['author_names','paper_title', 'year']
dblp_hapy = hash_blocking(dblp, blocking_columns)
acm_hapy = hash_blocking(acm,blocking_columns)
sim_hapy = apply_similarity_blocks(dblp_hapy, acm_hapy, threshold, jaccard_similarity, set_indices)
print(len(sim_hapy))


blocking_columns = ['author_names', 'publication_venue','year']
dblp_hapvy = hash_blocking(dblp, blocking_columns)
acm_hapvy = hash_blocking(acm,blocking_columns)
sim_hapvy = apply_similarity_blocks(dblp_hapvy, acm_hapvy, threshold, jaccard_similarity, set_indices)
print(len(sim_hapvy))


blocking_columns = ['author_names','paper_title', 'publication_venue','year']
dblp_happy = hash_blocking(dblp, blocking_columns)
acm_happy = hash_blocking(acm,blocking_columns)
sim_happy = apply_similarity_blocks(dblp_happy, acm_happy, threshold, jaccard_similarity, set_indices)
print(len(sim_happy))


blocking_columns = ['paper_title', 'publication_venue']
dblp_hppv = hash_blocking(dblp, blocking_columns)
acm_hppv = hash_blocking(acm,blocking_columns)
sim_hppv = apply_similarity_blocks(dblp_hppv, acm_hppv, threshold, jaccard_similarity, set_indices)
print(len(sim_hppv))


blocking_columns = ['paper_title', 'year']
dblp_hpy = hash_blocking(dblp, blocking_columns)
acm_hpy = hash_blocking(acm,blocking_columns)
sim_hpy = apply_similarity_blocks(dblp_hpy, acm_hpy, threshold, jaccard_similarity, set_indices)



blocking_columns = ['paper_title', 'publication_venue','year']
dblp_hppvy = hash_blocking(dblp, blocking_columns)
acm_hppvy = hash_blocking(acm,blocking_columns)
sim_hppvy = apply_similarity_blocks(dblp_hppvy, acm_hppvy, threshold, jaccard_similarity, set_indices)



Processing time: 9.235383987426758 seconds. Number of similar pairs: 1099
1099
Processing time: 7.105170249938965 seconds. Number of similar pairs: 2517
2517
Processing time: 7.793699026107788 seconds. Number of similar pairs: 1485
1485
Processing time: 9.74803614616394 seconds. Number of similar pairs: 1096
1096
Processing time: 8.959939002990723 seconds. Number of similar pairs: 1067
1067
Processing time: 7.684653043746948 seconds. Number of similar pairs: 1375
1375
Processing time: 8.15241289138794 seconds. Number of similar pairs: 1065
1065
Processing time: 7.8996360301971436 seconds. Number of similar pairs: 1692
1692
Processing time: 8.138370037078857 seconds. Number of similar pairs: 1518
Processing time: 8.150041818618774 seconds. Number of similar pairs: 1516


In [ ]:
import hashlib

def ngrams_tuple(s, n):
    return [tuple(s[i:i+n]) for i in range(len(s)-n+1)]

def hash_ngram_values(ngram_values):
    return hashlib.sha256(str(hash(tuple(sorted(ngram_values.items())))).encode()).hexdigest()

def n_gram_blocking(dataframe, columns, n):
    # n-gram for each selected column 
    for column in columns:
        key_name = f'ngram_key_{column}'
        dataframe[key_name] = dataframe[column].apply(lambda x: ''.join(map(str, x)))

        ngram_col_name = f'ngram_values_{column}'
        dataframe[ngram_col_name] = dataframe[key_name].apply(lambda x: tuple(ngrams_tuple(x, n)))

    # block them and each block consists of n-gram values and the respective index (id) 
    blocks = {}
    for index, row in dataframe.iterrows():
        ngram_values = {column: row[f'ngram_values_{column}'] for column in columns}
        hashable_ngram_values = hash_ngram_values(ngram_values)
        
        if hashable_ngram_values in blocks:
            blocks[hashable_ngram_values]['index'].add(row['index'])
        else:
            blocks[hashable_ngram_values] = {'index': {row['index']}, **ngram_values}

    return blocks


dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)




Die Methode braucht nur die jeweiligen Match methoden: Verwende eins als Baseline und das andere als Comparison. Müssten uns für eine Match methode entscheiden lass z.B sagen similarity_appvy (a = author, p = paper_title, pv= publication_venue, y = year) als Baseline und vergleichbare Matching-Methoden (alle anderen initials similarities) evaluieren. Natürlich kann man auch die gleiche Methode, aber mit anderen n (n-gram) wert und treshold vergleichen. Müssten uns für eine Baseline entscheiden?

In [ ]:
def evaluate_similarity(baseline, comparison):
    baseline_set, comparison_set = set(baseline), set(comparison)

    tp = len(baseline_set.intersection(comparison_set))
    fp = len(comparison_set - baseline_set)
    fn = len(baseline_set - comparison_set)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    f_measure = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0


    return precision, recall, f_measure

print(evaluate_similarity(sim_appvy, sim_ap))



(0.9779785431959345, 0.8686058174523571, 0.9200531208499335)


ToDo = - Eventuell weitere Blocking Methoden
       - kombinieren mit block ranges, welches Dataframe nach Jahren und publisher aufteilt in blöcke und dann jedes der Blöcke zusätlich mit einem anderen blocking oder direkt vergelichen
       - Aufgabe 2.3) die gewählte Baseline als CSV und kmeans darauf !

In [21]:
def apply_similarity(df1, df2, threshold, selected_columns, similarity_function):
    similar_pairs = []

    for index1, row1 in df1.iterrows():
        for index2, row2 in df2.iterrows():
            average_similarity = 0.0

            for col_df1, col_df2 in zip(selected_columns, selected_columns):
                value_df1 = set(row1[col_df1])
                value_df2 = set(row2[col_df2])

                similarity = similarity_function(value_df1, value_df2)
                average_similarity += similarity

            if len(selected_columns) > 1:
                average_similarity /= len(selected_columns)
            if average_similarity >= threshold:

                pair = (row1['index'], row2['index'])
                similar_pairs.append(pair)
    return similar_pairs

dblp_csv = 'CSV-files/dblp.csv'
dblp = pd.read_csv(dblp_csv)
acm_csv = 'CSV-files/acm.csv'
acm = pd.read_csv(acm_csv)
dblp['year'] = dblp['year'].astype(str)
acm['year'] = acm['year'].astype(str)
selected_columns = ['author_names','paper_title', 'year']

pairs = apply_similarity(dblp, acm, 0.7 ,selected_columns, sim.jaccard_similarity)
print(len(pairs))

651363


In [19]:
def apply_similarity_all_columns(df1, df2, threshold, similarity_function):
    similar_pairs = []

    # Reset indices to ensure a clean comparison
    df1_reset = df1.reset_index(drop=True)
    df2_reset = df2.reset_index(drop=True)

    # Obtain all column names from both DataFrames
    all_columns = set(df1_reset.columns).intersection(df2_reset.columns)

    for index1, row1 in df1_reset.iterrows():
        for index2, row2 in df2_reset.iterrows():
            average_similarity = 0.0

            for col in all_columns:
                value_df1 = set(str(row1[col]).lower().split())
                value_df2 = set(str(row2[col]).lower().split())

                similarity = similarity_function(value_df1, value_df2)
                average_similarity += similarity

            if len(all_columns) > 0:
                average_similarity /= len(all_columns)

            if average_similarity >= threshold:
                pair = (index1, index2)
                similar_pairs.append(pair)

    return similar_pairs

pairs = apply_similarity_all_columns(dblp, acm, 0.7 , sim.jaccard_similarity)
print(len(pairs))

288


In [14]:
dblp['year'] = dblp['year'].astype(str)
acm['year'] = acm['year'].astype(str)
column_data_types = dblp.dtypes
print(column_data_types)
print(dblp['author_names'].apply(type).value_counts())
column_data_types = acm.dtypes
print(column_data_types)

paper_title          object
author_names         object
year                 object
publication_venue    object
index                object
dtype: object
author_names
<class 'str'>    2153
Name: count, dtype: int64
paper_title          object
author_names         object
year                 object
publication_venue    object
index                object
dtype: object
